In [6]:
%pip install requests
%pip install flask

import hashlib
import json
from time import time
import random
import requests
from flask import Flask, jsonify, request

Note: you may need to restart the kernel to use updated packages.



In [7]:
DIFFICULTY = 4

In [8]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.nodes = set()

        # Create the genesis block
        self.new_block(previous_hash=1, proof=100)

    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
    
    @staticmethod
    def valid_proof(last_proof, proof):
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:DIFFICULTY] == '0' * DIFFICULTY
    
    @property
    def last_block(self):
        return self.chain[-1]
    
    def pow(self, last_proof):
        proof = random.randint(0, 2**64)
        while self.valid_proof(last_proof, proof) is False:
            proof = random.randint(0, 2**64)
        return proof
    
    def new_transaction(self, sender, recipient, amount):
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
            'timestamp': time(),
        })
        return self.last_block['index'] + 1
    
    def new_block(self, proof, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'nonce': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transactions = []
        self.chain.append(block)
        return block
    
    def valid_chain(self, chain):
        last_block = chain[0]
        current_index = 1
        while current_index < len(chain):
            block = chain[current_index]
            if block['previous_hash'] != self.hash(last_block):
                return False
            if not self.valid_proof(last_block['nonce'], block['nonce']):
                return False
            last_block = block
            current_index += 1
        return True


In [9]:
blockchain = Blockchain()
my_ip = '0.0.0.0'
my_port = 5000
node_identifier = f'node_{my_port}'
mine_owner = 'master'
mine_profit = 0.1

In [5]:
app = Flask(__name__)

@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!")
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return jsonify(response), 200

@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json()
    print("transactions_new!!! : ", values)
    required = ['sender', 'recipient', 'amount']
    if not all(k in values for k in required):
        return 'missing values', 400
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])
    response = {'message': 'Transaction will be added to Block {%s}' % index}
    return jsonify(response), 201

@app.route('/mine', methods=['GET'])
def mine():
    print("MINING STARTED")
    last_block = blockchain.last_block
    last_proof = last_block['nonce']
    proof = blockchain.pow(last_proof)
    blockchain.new_transaction(
        sender=mine_owner,
        recipient=node_identifier,
        amount=mine_profit  # coinbase transaction
    )
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)
    print("MINING FINISHED")
    response = {
        'message': 'new block found!',
        'index': block['index'],
        'transactions': block['transactions'],
        'nonce': block['nonce'],
        'previous_hash': block['previous_hash']
    }
    return jsonify(response), 200

if __name__ == '__main__':
    app.run(host=my_ip, port=my_port)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.30.1.62:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Oct/2024 02:16:38] "GET /chain HTTP/1.1" 200 -


chain info requested!


127.0.0.1 - - [03/Oct/2024 02:16:40] "POST /transactions/new HTTP/1.1" 201 -


transactions_new!!! :  {'sender': 'test_from', 'recipient': 'test_to', 'amount': 3}


...

chain info requested!


127.0.0.1 - - [03/Oct/2024 02:23:52] "GET /chain HTTP/1.1" 200 -


chain info requested!


127.0.0.1 - - [03/Oct/2024 02:31:20] "GET /chain HTTP/1.1" 200 -


chain info requested!
